# Simulation Tests

This notebook is used to benchmark scanpro methods using simulated cell counts. Cell counts are simulated using the hierarchical model described in the propeller paper (Phipson et al., 2022). 

1 & 2: Testing performance of scanpro on 100 random datasets with no differences (1) and with differences (2). The hit rate is the proportion of tests where all clusters were identified as non-significant (1) or where only cardiomyocytes, fibroblasts and smooth muscle cells are identified as significant (2).

In [1]:
import pandas as pd
import numpy as np
import multiprocessing as mp
import time

from scanpro import scanpro
from scanpro.utils import simulate_cell_counts, simulate_cell_counts_2, convert_counts_to_df, estimate_params_from_counts

In [2]:
OUT_PATH = './results/benchmark/'

In [3]:
def simulate_data(p, a, b, n_reps, n_conds=2, n=20, mu=5000, n_sims=100, null=True):
    if b is None:
        b = a * (1-p) / p
        
    datasets = []
    for sim in range(n_sims):
        if null:  # null simulation -> no differences
            counts = simulate_cell_counts(p, n_reps, a, b, n_conds, n=n, mu=mu)
        else:  # true differences
            counts = simulate_cell_counts_2(p, n_reps, a, b, n_conds, n=n, mu=mu)

        counts_df = convert_counts_to_df(counts, column_name="cluster")

        datasets.append(counts_df)
    
    return datasets

# Benchmarking

In [4]:
def monitor_jobs(jobs):
    """
    Monitor the status of jobs submitted to a pool.

    Parameters
    ----------
    jobs : list of job objects
        List of job objects, e.g. as returned by pool.map_async().
    """

    if isinstance(jobs, dict):
        jobs = list(jobs.values())

    from tqdm import tqdm_notebook as tqdm
    pbar = tqdm(total=len(jobs))
    
    # Wait for all jobs to finish
    n_ready = sum([job.ready() for job in jobs])
    while n_ready != len(jobs):
        if n_ready != pbar.n:
            pbar.n = n_ready
            pbar.refresh()
        time.sleep(1)
        n_ready = sum([job.ready() for job in jobs])

    pbar.n = n_ready  # update progress bar to 100%
    pbar.refresh()
    pbar.close()

In [5]:
def test_performance(datasets,
                     n_reps,  # number of samples per condition
                     transform,
                     repd_data=False
                     ):
    """Test the performance of normal and bootstrap scanpro on simulated data.
    :param list datasets: List of datasets as pandas dataframes to run scanpro on
    :param int n_reps: Number of replicates the bootstrap is going to generate
    :param str transform: method of transformation (logit or arcsin)
    :return pandas.DataFrame all_run_results: A dataframe with results from all runs.
    """
    
    pool = mp.Pool()
    jobs = []

    for dataset in datasets:

        if not repd_data:
            samples_col = None
        else:
            samples_col = "sample"

        # Run Scanpro or scanpro bootstrapping on 100 datasets
        job = pool.apply_async(scanpro, (dataset,), dict(clusters_col="cluster", conds_col="group", 
                                                                 samples_col=samples_col, n_reps=n_reps,
                                                                 transform=transform, verbosity=0))
        jobs.append(job)

    pool.close()
    monitor_jobs(jobs)

    results = [job.get() for job in jobs]
    pool.join()

    # Add ID to each result
    for i, result in enumerate(results):
        result.results["run"] = i + 1
        
    # Collect result
    all_run_results = pd.concat([result.results for result in results])

    return all_run_results

-----------

## 1. Null simulations

### Simulate datasets

In [6]:
np.random.seed(10)

p = np.array([0.01, 0.05, 0.15, 0.34, 0.45])  # clusters proportions in all samples
a = 10
b = a*(1-p)/p
n_reps_list = [4, 8, 10, 20]

datasets_null = {f"{n_reps}_reps": simulate_data(p, a, b, n_reps, n_conds=2, n=20, mu=5000, n_sims=100, null=True) for n_reps in n_reps_list}

### Run scanpro on datasets

In [7]:
null_results = {}
for n_reps in n_reps_list:
    for replicated in [True, False]:
        for transform in ["logit", "arcsin"]:
            print(f"n_reps={n_reps} | replicated={replicated} | transform={transform}")
            data = datasets_null[f"{n_reps}_reps"]
            result = test_performance(data, transform=transform,
                                      n_reps=n_reps, repd_data=replicated)
            
            result["n_reps"] = n_reps
            result["replicated"] = "rep" if replicated else "norep"
            result["transform"] = transform
            
            null_results[(n_reps, replicated, transform)] = result

n_reps=4 | replicated=True | transform=logit


  0%|          | 0/100 [00:00<?, ?it/s]

n_reps=4 | replicated=True | transform=arcsin


  0%|          | 0/100 [00:00<?, ?it/s]

n_reps=4 | replicated=False | transform=logit


...

  0%|          | 0/100 [00:00<?, ?it/s]

n_reps=20 | replicated=False | transform=logit


  0%|          | 0/100 [00:00<?, ?it/s]

n_reps=20 | replicated=False | transform=arcsin


  0%|          | 0/100 [00:00<?, ?it/s]

### Join results

In [8]:
null_table = pd.concat(null_results.values())

In [9]:
null_table["correct"] = null_table["p_values"] > 0.05  # None of the pvalues should be significant

In [10]:
null_table.to_csv(OUT_PATH + "simulation_results_null.tsv", sep="\t")

In [11]:
null_table.head()

,baseline_props,mean_props_cond_1,mean_props_cond_2,prop_ratio,t_statistics,p_values,adjusted_p_values,run,n_reps,replicated,transform,correct
clusters,,,,,,,,,,,,
c1,0.010889,0.008927,0.012071,0.739575,-1.246163,0.222348,0.902555,1,4,rep,logit,True
c2,0.040336,0.039698,0.039234,1.011825,0.123475,0.902555,0.902555,1,4,rep,logit,True
c3,0.151057,0.148428,0.156345,0.949361,-0.271865,0.787588,0.902555,1,4,rep,logit,True
c4,0.365765,0.357589,0.377591,0.947028,-0.406381,0.687347,0.902555,1,4,rep,logit,True
c5,0.431953,0.445358,0.414759,1.073774,0.625436,0.536414,0.902555,1,4,rep,logit,True


---------

## 2. Simulation with true differences

### Data preperation
see https://phipsonlab.github.io/propeller-paper-analysis/SimTrueDiff.html

In [12]:
heart_counts = pd.read_csv('data/heart_counts.tsv', sep='\t')
heart_counts.drop(['Condition', 'Sex'], inplace=True, axis=1)
heart_counts = heart_counts.set_index('Sample').T
heart_counts.drop('Erythroid', inplace=True)  # remove erythroids

# proportions of each cluster in all samples
true_props = heart_counts.sum(axis=1) / heart_counts.sum(axis=1).sum()  # sum of cells in cluster / sum of all cells
true_props = true_props.to_frame(name="props")

# estimate beta paramters from counts
params = estimate_params_from_counts(heart_counts)  # rows are clusters
a = params[1]
b = params[2]

# Set up true proportions for the two groups
grp1_trueprops = true_props.values.flatten()
grp2_trueprops = true_props.values.flatten()

grp2_trueprops[0] = grp1_trueprops[0]/2
grp2_trueprops[3] = grp2_trueprops[3]*2
grp2_trueprops[6] = grp1_trueprops[6]*3

grp2_trueprops[0] = grp2_trueprops[0] + (1-grp2_trueprops.sum())/2
grp2_trueprops[3] = grp2_trueprops[3] + (1-grp2_trueprops.sum())
 
# calculate beta for both groups
b1 = a*(1-grp1_trueprops)/grp1_trueprops
b2 = a*(1-grp2_trueprops)/grp2_trueprops
b_grps = [b1, b2]

b_grps

[Cardiomyocytes           1.769390
 Endothelial cells      107.760684
 Epicardial cells        47.297371
 Fibroblast              14.138098
 Immune cells            16.442437
 Neurons                143.134155
 Smooth muscle cells    458.574736
 dtype: float64,
 Cardiomyocytes           4.742325
 Endothelial cells      107.760684
 Epicardial cells        47.297371
 Fibroblast               4.666523
 Immune cells            16.442437
 Neurons                143.134155
 Smooth muscle cells    150.405074
 dtype: float64]

### Simulate datasets

In [13]:
np.random.seed(10)

n_reps_list = [4, 8, 10, 20]

datasets_difs = {f"{n_reps}_reps": simulate_data(true_props, a, b_grps, n_reps, n_conds=2, n=20, mu=5000, n_sims=100, null=False) for n_reps in n_reps_list}

### Run Scanpro

In [14]:
dif_results = {}
for n_reps in n_reps_list:
    for replicated in [True, False]:
        for transform in ["logit", "arcsin"]:
            print(f"n_reps={n_reps} | replicated={replicated} | transform={transform}")
            data = datasets_difs[f"{n_reps}_reps"]
            result = test_performance(data, transform=transform,
                                      n_reps=n_reps, repd_data=replicated)
            
            result["n_reps"] = n_reps
            result["replicated"] = "rep" if replicated else "norep"
            result["transform"] = transform
            
            dif_results[(n_reps, replicated, transform)] = result

n_reps=4 | replicated=True | transform=logit


  0%|          | 0/100 [00:00<?, ?it/s]

n_reps=4 | replicated=True | transform=arcsin


  0%|          | 0/100 [00:00<?, ?it/s]

n_reps=4 | replicated=False | transform=logit


...

  0%|          | 0/100 [00:00<?, ?it/s]

n_reps=20 | replicated=False | transform=logit


  0%|          | 0/100 [00:00<?, ?it/s]

n_reps=20 | replicated=False | transform=arcsin


  0%|          | 0/100 [00:00<?, ?it/s]

### Join results

In [15]:
dif_table = pd.concat(dif_results.values())

In [16]:
# Estimate whether calls are correct - clusters 0, 3 and 6 should be different
differential_groups = b_grps[0].iloc[[0,3,6]].index.tolist()
differential_groups

['Cardiomyocytes', 'Fibroblast', 'Smooth muscle cells']

In [17]:
# Assign whether p-values are correct
dif_table["correct"] = [row["p_values"] < 0.05 if row["clusters"] in differential_groups else row["p_values"] >= 0.05 for i, row in dif_table.reset_index().iterrows()]

In [18]:
dif_table.to_csv(OUT_PATH + "simulation_results_diff.tsv", sep="\t")

In [19]:
dif_table.head()

,baseline_props,mean_props_cond_1,mean_props_cond_2,prop_ratio,t_statistics,p_values,adjusted_p_values,run,n_reps,replicated,transform,correct
clusters,,,,,,,,,,,,
Cardiomyocytes,0.459120,0.560036,0.316807,1.767752,3.855004,0.003372,0.007869,1,4,rep,logit,True
Endothelial cells,0.078884,0.076970,0.079162,0.972315,-0.172792,0.866368,0.982673,1,4,rep,logit,True
Epicardial cells,0.056030,0.057622,0.060288,0.955778,-0.022285,0.982673,0.982673,1,4,rep,logit,True
Fibroblast,0.319075,0.211927,0.454100,0.466697,-4.429626,0.001374,0.004808,1,4,rep,logit,True
Immune cells,0.058848,0.075221,0.045988,1.635662,0.936577,0.371713,0.650498,1,4,rep,logit,True
